---

_You are currently looking at **version 1.0** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-text-mining/resources/d9pwm) course resource._

---

# Assignment 1

In this assignment, you'll be working with messy medical data and using regex to extract relevant infromation from the data. 

Each line of the `dates.txt` file corresponds to a medical note. Each note has a date that needs to be extracted, but each date is encoded in one of many formats.

The goal of this assignment is to correctly identify all of the different date variants encoded in this dataset and to properly normalize and sort the dates. 

Here is a list of some of the variants you might encounter in this dataset:
* 04/20/2009; 04/20/09; 4/20/09; 4/3/09
* Mar-20-2009; Mar 20, 2009; March 20, 2009;  Mar. 20, 2009; Mar 20 2009;
* 20 Mar 2009; 20 March 2009; 20 Mar. 2009; 20 March, 2009
* Mar 20th, 2009; Mar 21st, 2009; Mar 22nd, 2009
* Feb 2009; Sep 2009; Oct 2010
* 6/2008; 12/2009
* 2009; 2010

Once you have extracted these date patterns from the text, the next step is to sort them in ascending chronological order accoring to the following rules:
* Assume all dates in xx/xx/xx format are mm/dd/yy
* Assume all dates where year is encoded in only two digits are years from the 1900's (e.g. 1/5/89 is January 5th, 1989)
* If the day is missing (e.g. 9/2009), assume it is the first day of the month (e.g. September 1, 2009).
* If the month is missing (e.g. 2010), assume it is the first of January of that year (e.g. January 1, 2010).

With these rules in mind, find the correct date in each note and return a pandas Series in chronological order of the original Series' indices.

For example if the original series was this:

    0    1999
    1    2010
    2    1978
    3    2015
    4    1985

Your function should return this:

    0    2
    1    4
    2    0
    3    1
    4    3

Your score will be calculated using [Kendall's tau](https://en.wikipedia.org/wiki/Kendall_rank_correlation_coefficient), a correlation measure for ordinal data.

*This function should return a Series of length 500 and dtype int.*

In [1]:
import pandas as pd

doc = []
with open('dates.txt') as file:
    for line in file:
        doc.append(line)

df = pd.Series(doc)
df.head(10)

0         03/25/93 Total time of visit (in minutes):\n
1                       6/18/85 Primary Care Doctor:\n
2    sshe plans to move as of 7/8/71 In-Home Servic...
3                7 on 9/27/75 Audit C Score Current:\n
4    2/6/96 sleep studyPain Treatment Pain Level (N...
5                    .Per 7/06/79 Movement D/O note:\n
6    4, 5/18/78 Patient's thoughts about current su...
7    10/24/89 CPT Code: 90801 - Psychiatric Diagnos...
8                         3/7/86 SOS-10 Total Score:\n
9             (4/10/71)Score-1Audit C Score Current:\n
dtype: object

In [2]:
def date_sorter():
    #import package
    import re
    import datetime
    import numpy as np
    #split data into two group, first group operate on full number date,second operate on missing or letter date
    #operate on first set
    date_dict = {'year':[],'month':[],'day':[]}
    lists1 = []
    for i in range(125):
        data = re.findall(r'\d{1,2}[/-]\d{1,2}[/-]\d{2,4}',df[i])
        if len(data)!=0:
            lists1.append(data)

    for i in lists1:#extract year,month, day for each data of first set
        if '/' in i[0]:
            month, day, year = i[0].split('/')#split by / and extract year month day
        elif '-' in i: 
            month, day, year = i[0].split('-')
        if len(year) == 2:
            year = '19'+year
        date_dict['year'].append(year)
        date_dict['month'].append(month)
        date_dict['day'].append(day)
    #define dict to correlate letter month and number month
    month_dict={'Jan':'01','Feb':'02','Mar':'03','Apr':'04','May':'05','Jun':'06','Jul':'07','Aug':'08','Sep':'09','Oct':'10','Nov':'11','Dec':'12','Decemeber':'12',
          'January':'01','Janaury':'01','February':'02','March':'03','April':'04','May':'05','June':'06','July':'07','August':'08','September':'09','October':'10','November':'11','December':'12'}
    #operate on second set
    lists = []
    for i in range(125,500):#extract year,month, day for each data of second set
        data = re.findall(r'(?:(?:\d{1,2} )?(?:Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)[a-z]*[ -,.]*)?(?:\d{1,2}[,. ]*)?(?:\d{1,2}/)?\d{4}?',df[i])
        if len(data)!=0:
            lists.append(data)
    for i in range(len(lists)):#preprocess: remove minor mistake
        if len(lists[i]) > 1:
            lists[i] = [lists[i][0]]
        if i > len(lists) - 50 and len(lists[i][0])>4:
            lists[i] = [lists[i][0][-4:]]
        if '/' in lists[i][0] and ' ' in lists[i][0]:
            lists[i] = [lists[i][0].split(' ')[1]]
            
    date_dict2 = date_dict
    for i in lists:
        i[0] =  re.sub(r'[,.]', '', i[0])#remove , and .
        if i[0]=='7787': i[0]='2008'#fix one error
        if len(i[0].split(' ')) == 3:#if no missing
            items = i[0].split(' ')
            for item in items:
                if item in month_dict:
                    date_dict2['month'].append(month_dict[item])
                elif len(item) <= 2:
                    date_dict2['day'].append(item)
            date_dict2['year'].append(i[0].split(' ')[2])
        elif len(i[0].split(' ')) == 2:#if missing one value
            month, year = i[0].split(' ')
            date_dict2['month'].append(month_dict[month])
            date_dict2['year'].append(year)
            date_dict2['day'].append('01')
        else:#if missing two value
            if '/' in i[0]:
                month, year = i[0].split('/')
                date_dict2['month'].append(month)
                date_dict2['year'].append(year)
                date_dict2['day'].append('01')
            else:
                date_dict2['month'].append('01')
                date_dict2['year'].append(i[0])
                date_dict2['day'].append('01')
                
    for key in ['month','day']:#change day and month to two digit : 1->01,otherwise will make mistake
        for i in range(500):
            if len(date_dict2[key][i]) < 2:
                date_dict2[key][i] = '0' + date_dict2[key][i]
    #change string to date            
    date = [date_dict2['year'][i]+'-'+date_dict2['month'][i]+'-'+date_dict2['day'][i] for i in range(500) ]
    sorted_date = np.argsort(np.array(date)).argsort()

    return pd.Series(sorted_date)

In [3]:
#date_sorter()

0      272
1      169
2        2
3       54
4      312
5       98
6       87
7      228
8      179
9        0
10     167
11      48
12     334
13       7
14     236
15     458
16     129
17      67
18     329
19      75
20     332
21     103
22     234
23      60
24     158
25     157
26     227
27      65
28       4
29     191
      ... 
470    140
471    339
472    446
473     45
474      5
475    488
476    215
477    279
478    269
479    310
480    478
481     26
482    230
483    292
484    388
485    196
486     15
487    254
488     69
489    163
490    416
491    439
492    176
493     80
494    372
495     95
496    408
497    431
498    402
499    105
Length: 500, dtype: int64